In [ ]:
num = 1 

In [6]:
!pip install rank_bm25
!pip install underthesea
!pip install gdown

In [14]:
import gdown

file_id1 = '1hUPYt7_8J6MtuyIo60Dv4MCZ04IBBrq3'
file_id2 = '1AhEOx0cvq7zL67sHgApD_3ji40zNSxIT'
file_id3 = '1uutCAEumawSqFMRD292n4cYNxqZJNPMf'

url1 = f'https://drive.google.com/uc?id={file_id1}'
url2 = f'https://drive.google.com/uc?id={file_id2}'
url3 = f'https://drive.google.com/uc?id={file_id3}'

output1 = 'question_answer.json'
output2 = 'legal_dict.json'
output3 = 'bm25_model.pkl'

gdown.download(url1, output1, quiet=False)
gdown.download(url2, output2, quiet=False)
gdown.download(url3, output3, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1hUPYt7_8J6MtuyIo60Dv4MCZ04IBBrq3
From (redirected): https://drive.google.com/uc?id=1hUPYt7_8J6MtuyIo60Dv4MCZ04IBBrq3&confirm=t&uuid=19c34947-a38c-4ce4-9aaa-2e866ee5bbaf
To: /kaggle/working/question_answer.json
100%|██████████| 132M/132M [00:00<00:00, 168MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1AhEOx0cvq7zL67sHgApD_3ji40zNSxIT
From (redirected): https://drive.google.com/uc?id=1AhEOx0cvq7zL67sHgApD_3ji40zNSxIT&confirm=t&uuid=2e9e04b0-46de-4c67-9f06-53016de4d3a4
To: /kaggle/working/legal_dict.json
100%|██████████| 395M/395M [00:02<00:00, 189MB/s]  
Downloading...
From (original): https://drive.google.com/uc?id=1uutCAEumawSqFMRD292n4cYNxqZJNPMf
From (redirected): https://drive.google.com/uc?id=1uutCAEumawSqFMRD292n4cYNxqZJNPMf&confirm=t&uuid=7378483f-b2c1-476f-aa38-fb51f3c6c6d0
To: /kaggle/working/bm25_model.pkl
100%|██████████| 232M/232M [00:01<00:00, 220MB/s] 


'bm25_model.pkl'

In [15]:
import json
import csv
import string
import pickle
import numpy as np
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from underthesea import word_tokenize

In [16]:

number = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
chars = ["a", "b", "c", "d", "đ", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o"]
stop_word = number + chars + ["của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
            "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
            "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
            "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
            "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
            "như", "đó", "mà", "nơi", "”", "“"]

def remove_stopword(w):
    return w not in stop_word

def remove_punctuation(w):
    return w not in string.punctuation

def lower_case(w):
    return w.lower()

def bm25_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = list(map(lower_case, tokens))
    tokens = list(filter(remove_punctuation, tokens))
    tokens = list(filter(remove_stopword, tokens))
    return tokens


In [18]:

with open('/kaggle/working/question_answer.json', 'r', encoding='utf-8') as f:
    question_answer_data = json.load(f)

with open('/kaggle/working/legal_dict.json', 'r', encoding='utf-8') as f:
    legal_dict = json.load(f)
law_texts = [entry['text'] for entry in legal_dict.values()]

with open('/kaggle/working/bm25_model.pkl', 'rb') as f:
    bm25 = pickle.load(f)

In [21]:

chunk_size = len(question_answer_data) // 8
chunks = [question_answer_data[i:i + chunk_size] for i in range(0, len(question_answer_data), chunk_size)]

chunk_to_process = chunks[num - 1] 


In [22]:

output = []

print(f"Processing part {num}...")
for qa in tqdm(chunk_to_process, desc=f"Processing part {num}"):
    question = qa['question']
    answers = [answer['text'] for answer in qa['answer']]
    query_tokens = bm25_tokenizer(question)
    scores = bm25.get_scores(query_tokens)
    ranked_indices = np.argsort(scores)[::-1]  
    
    negative_indices = []
    for idx in ranked_indices:
        law_key = list(legal_dict.keys())[idx]
        
        if legal_dict[law_key]['text'] not in answers:
            negative_indices.append(idx)
        
        if len(negative_indices) == 20:
            break
    
    negatives = [legal_dict[list(legal_dict.keys())[idx]]['text'] for idx in negative_indices]
    
    output.append({
        'query': question, 'pos': answers, 'neg': negatives, 'prompt': "Given a question, retrieve legal passages that answer the question", 'type': "normal"
    })

with open(f'output_part{num}.jsonl', 'w', encoding='utf-8') as f:
    for item in output:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"Đã xử lý và lưu phần {num} vào file 'output_part{num}.jsonl'")

Processing part 1...


Processing part 1:   0%|          | 7/10402 [00:12<5:12:54,  1.81s/it]


KeyboardInterrupt: 

In [ ]:
import zipfile

zip_filename = f'output_part{num}.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(f'output_part{num}.jsonl', arcname=f'output_part{num}.jsonl')

print(f"Đã zip file thành '{zip_filename}'")
